In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
%cd /content/
%rm -rf pyscribe
!git clone https://github.com/w1nsid/pyscribe
%cd pyscribe
!git pull 

In [ ]:
%pip install ffmpeg-python
%pip install openai
%pip install git+https://github.com/openai/whisper.git

In [ ]:
import classes as pyscribe
import logging
from pathlib import Path
import time

source_folder = "/content/drive/MyDrive/pyscribe/input"
output_folder = "/content/drive/MyDrive/pyscribe/out"
model = "large-v2"
language = "french"

conf = pyscribe.Config("config.json")
conf.source_folder = source_folder
conf.output_folder = output_folder
conf.model = model
conf.language = language
conf.load("key.json")

conf.print_config()

In [ ]:
pyscribe.setup_logging()

logging.info("This is an info message.")
logging.error("This is an error message.")
logging.warning("This is a warning message.")

# Init Whisperer
whisperer = pyscribe.Whisperer(conf)
# Init gpt
gpt = pyscribe.ChatGPTHAndler(conf)

In [ ]:
source_folder = Path(conf.source_folder)

# get list of videos in source folder, extension .mp4 mkv
videos = [
    f for f in source_folder.iterdir() if f.is_file() and f.suffix in [".mp4", ".mkv"]
]
logging.info(f"Found {len(videos)} videos to process")


# read context from file
context_file = Path("/content/drive/MyDrive/pyscribe/context.txt")
context = context_file.read_text() if context_file.exists() else None

In [ ]:
# Process videos
for video in videos:
    tic = time.time()
    logging.info(f"Processing video {video.name}")
    video_obj = pyscribe.Media(path=video, config=conf, context=context, save_raw=True)
    video_obj.generate_subtitles(whisperer)
    gpt.process_media(video_obj)
    logging.info(f"Processed video {video.name} in {time.time() - tic:.1f} sec")